In [4]:
import main
from os import listdir
import pandas as pd
import TAtT
import Christofides
import time as T
import tracemalloc

In [2]:

df = pd.DataFrame({'Nome':['a'], 'Tamanho':[0], 'BnB':[0], 'timeBnB':[0], 'spaceBnB':[0], 'TAtT':[0], 'timeTAtT':[0], 'spaceTAtT':[0], 'Christofides':[0], 'timeChristofides':[0], 'spaceChristofides':[0]})

In [6]:
import signal
import time as T
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


In [ ]:
files = list(listdir('ALL_tsp'))
dfTAtT = pd.DataFrame({'Name':['a'],'Val':[0],'time':[0],'space':[0]})
for file in files:
    try:
        with time_limit(60*30):
            name = file
            print(name)
            (G,_,_) = main.Graph('ALL_tsp/'+file)
            init = T.time()
            tracemalloc.start()
            snapshot1 = tracemalloc.take_snapshot()
            (_, W) = TAtT.TatT(G,1)
            snapshot2 = tracemalloc.take_snapshot()
            tracemalloc.stop()
            diferenca = snapshot2.compare_to(snapshot1, 'lineno')
            space = round(sum(stat.size_diff for stat in diferenca), 2)
            time = T.time() - init
            newrow = pd.DataFrame({'Name':[file],'Val':[W],'time':[round(time,2)],'space':[space]})
            dfTAtT = pd.concat([dfTAtT,newrow],ignore_index=True)
            dfTAtT.to_csv('CSV/TAtT.csv')
            print('add ',name,W,time,space)
    except TimeoutException as e:
        newroe = pd.DataFrame({'Name':[file],'Val':[None],'time':[30],'space':[None]})
        dfTAtT = pd.concat([dfTAtT,newrow],ignore_index=True)
        dfTAtT.to_csv('CSV/TAtT.csv')
        print("Timed out! ",file)
dfTAtT.drop([0],axis=0)


In [ ]:
files = list(listdir('ALL_tsp'))
dfChristofides = pd.DataFrame({'Name':['a'],'Val':[0],'time':[0],'space':[0]})
for file in files:
    try:
        name = file
        print(name)
        (G,_,_) = main.Graph('ALL_tsp/'+file)
        init = T.time()
        tracemalloc.start()
        snapshot1 = tracemalloc.take_snapshot()
        (_, W) = Christofides.Christofides(G,1)
        time = T.time() - init
        snapshot2 = tracemalloc.take_snapshot()
        tracemalloc.stop()      
        diferenca = snapshot2.compare_to(snapshot1, 'lineno')
        space = round(sum(stat.size_diff for stat in diferenca), 2)
        newrow = pd.DataFrame({'Name':[file],'Val':[W],'time':[time],'space':[space]})
        dfChristofides = pd.concat([dfChristofides,newrow],ignore_index=True)
        dfChristofides.to_csv('CSV/Christofides.csv')
        print('add ',name,W,time,space)
    except TimeoutException as e:
        newrow = pd.DataFrame({'Name':[file],'Val':[None],'time':[30],'space':[None]})
        dfChristofides = pd.concat([dfChristofides,newrow],ignore_index=True)
        dfTAtT.to_csv('CSV/TAtT.csv')
        print("Timed out! ",file)
dfChristofides.drop([0],axis=0)